In [1]:
!pip install Bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 2.2 MB/s eta 0:00:0000:0100:01m


In [2]:
!pip install modlamp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 2.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.2/345.2 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.2/397.2 kB 4.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for modlamp: filename=modlamp-4.3.0-py3-none-any.whl size=173889 sha256=6adcff4fac8fc660d9f282b12ad660acd9c24ea8d6cf8e43c64c749542d8eeeb
  Stored in directory: /Users/ggcl7/Library/Caches/pip/wheels/fc/09/85/aefb43b4f79b6323733e8276c157c4851409b65f77d7248ace
Successfully built modlamp


In [3]:
!pip install tqdm

In [1]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis as PA
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor
from sklearn.model_selection import train_test_split
import pandas as pd
import os, re, math, platform
from tqdm import tqdm
from collections import Counter
import numpy as np
from Bio import Seq, SeqIO

In [2]:
# Miscellaneous
_AALetter = ['A', 'C', 'D', 'E', 'F', 'G', 'H',
             'I', 'K', 'L', 'M', 'N', 'P', 'Q',
             'R', 'S', 'T', 'V', 'W', 'Y']


"""
n_gram statistics
"""

_AALetter = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 
            'I', 'K', 'L', 'M', 'N', 'P', 'Q', 
            'R', 'S', 'T', 'V', 'W', 'Y']

def get_aan_corpus(n=2):
    '''
    Get AA corpus of n_gram (e.g. Di, Tri, etc.)
    Output: AA n_gram corpus ((e.g. Di:400, Tri:3000, etc.))
    '''
    n_corpus = []
    if n <= 2:
        for i in _AALetter:
            for j in _AALetter:
               n_corpus.append("{}{}".format(i, j))
        return n_corpus
    for i in get_aan_corpus(n - 1):
        for j in _AALetter:
            n_corpus.append("{}{}".format(i, j))
    return n_corpus


def get_ngram_counts(seq, n=2):
    '''
    Get n_gram statistics
    Input: peptide sequence and n
    Ouput: n_gram statistic (dictionary) {A.A Corp: Counts}
    '''
    # Get the name of ngram feature
    if n == 2:
        prefix = 'DPC'
    elif n == 3:
        prefix = 'TPC'
    else:
        prefix = '{}gram'.format(n)

    ngrams = [seq[i: i + n] for i in range(len(seq) - n + 1)]
    n_corpus = get_aan_corpus(n)
    ngram_stat = {}
    for aa_ng in n_corpus:
        ngram_stat['{}|{}'.format(prefix, aa_ng)] = ngrams.count(aa_ng) / len(ngrams) * 100
    return ngram_stat


def minSequenceLength(fastas):
    minLen = 10000
    for i in fastas:
        if minLen > len(i[1]):
            minLen = len(i[1])
    return minLen


def minSequenceLengthWithNormalAA(fastas):
    minLen = 10000
    for i in fastas:
        if minLen > len(re.sub('-', '', i[1])):
            minLen = len(re.sub('-', '', i[1]))
    return minLen


"""
    input.fasta:      the input protein sequence file in fasta format.
    k_space:          the gap of two amino acids, integer, defaule: 5
    output:           the encoding file, default: 'encodings.tsv'
"""


def generateGroupPairs(groupKey):
    gPair = {}
    for key1 in groupKey:
        for key2 in groupKey:
            gPair['CKSAAGP|'+key1+'.'+key2] = 0
    return gPair



def Rvalue(aa1, aa2, AADict, Matrix):
    return sum([(Matrix[i][AADict[aa1]] - Matrix[i][AADict[aa2]]) ** 2 for i in range(len(Matrix))]) / len(Matrix)


def paac(fastas, lambdaValue=30, w=0.05, **kw):
    if minSequenceLengthWithNormalAA(fastas) < lambdaValue + 1:
        print('Error: all the sequence length should be larger than the lambdaValue+1: ' + str(lambdaValue + 1) + '\n\n')
        return 0

    dataFile = re.sub('codes$', '', os.path.split(os.path.realpath('__file__'))[0]) + r'\data\PAAC.txt' if platform.system() == 'Windows' else re.sub('codes$', '', os.path.split(os.path.realpath('__file__'))[0]) + '/data/PAAC.txt'
    with open(dataFile) as f:
        records = f.readlines()
    AA = ''.join(records[0].rstrip().split()[1:])
    AADict = {}
    for i in range(len(AA)):
        AADict[AA[i]] = i
    AAProperty = []
    AAPropertyNames = []
    for i in range(1, len(records)):
        array = records[i].rstrip().split() if records[i].rstrip() != '' else None
        AAProperty.append([float(j) for j in array[1:]])
        AAPropertyNames.append(array[0])

    AAProperty1 = []
    for i in AAProperty:
        meanI = sum(i) / 20
        fenmu = math.sqrt(sum([(j-meanI)**2 for j in i])/20)
        AAProperty1.append([(j-meanI)/fenmu for j in i])

    encodings = []
    header = ['#']
    for aa in AA:
        header.append('PAAC|' + aa)
    for n in range(1, lambdaValue + 1):
        header.append('PAAC|lambda' + str(n))
    encodings.append(header)

    for i in fastas:
        name, sequence = i[0], re.sub('-', '', i[1])
        code = [name]
        theta = []
        for n in range(1, lambdaValue + 1):
            theta.append(
                sum([Rvalue(sequence[j], sequence[j + n], AADict, AAProperty1) for j in range(len(sequence) - n)]) / (
                len(sequence) - n))
        myDict = {}
        for aa in AA:
            myDict[aa] = sequence.count(aa)
        code = code + [myDict[aa] / (1 + w * sum(theta)) for aa in AA]
        code = code + [(w * j) / (1 + w * sum(theta)) for j in theta]
        encodings.append(code)
    return encodings






def insert_phycs(seq_df):
    seq_df = seq_df.copy()
    #  Function for compute Isoelectric Point or net_charge of peptide
    def get_ieq_nc(seq, is_iep=True):
        protparam = PA(seq)
        return protparam.isoelectric_point() if is_iep else protparam.charge_at_pH(7.0)

    # Calculating IsoElectricPoints and NeutralCharge
    data_size = seq_df.size
    seq_df['PHYC|IEP'] = list(map(get_ieq_nc, seq_df['Sequence'], [True] * data_size))  # IsoElectricPoints
    seq_df['PHYC|Net Charge'] = list(map(get_ieq_nc, seq_df['Sequence'], [False] * data_size))  # Charge(Neutral)

    # Calculating hydrophobic moment (My assume all peptides are alpha-helix)
    descrpt = PeptideDescriptor(seq_df['Sequence'].values, 'eisenberg')
    descrpt.calculate_moment(window=1000, angle=100, modality='max')
    seq_df['PHYC|Hydrophobic Moment'] = descrpt.descriptor.reshape(-1)

    # Calculating "Hopp-Woods" hydrophobicity
    descrpt = PeptideDescriptor(seq_df['Sequence'].values, 'hopp-woods')
    descrpt.calculate_global()
    seq_df['PHYC|Hydrophobicity'] = descrpt.descriptor.reshape(-1)

    # Calculating Energy of Transmembrane Propensity
    descrpt = PeptideDescriptor(seq_df['Sequence'].values, 'tm_tend')
    descrpt.calculate_global()
    seq_df['PHYC|Transmembrane Propensity'] = descrpt.descriptor.reshape(-1)

    # Calculating Aromaticity
    descrpt = GlobalDescriptor(seq_df['Sequence'].values)
    descrpt.aromaticity()
    seq_df['PHYC|Aromacity'] = descrpt.descriptor.reshape(-1)

    # Calculating Levitt_alpha_helical Propensity
    descrpt = PeptideDescriptor(seq_df['Sequence'].values, 'levitt_alpha')
    descrpt.calculate_global()
    seq_df['PHYC|Alpha Helical Propensity'] = descrpt.descriptor.reshape(-1)

    # Calculating Aliphatic Index
    descrpt = GlobalDescriptor(seq_df['Sequence'].values)
    descrpt.aliphatic_index()
    seq_df['PHYC|Aliphatic Index'] = descrpt.descriptor.reshape(-1)

    # Calculating Boman Index
    descrpt = GlobalDescriptor(seq_df['Sequence'].values)
    descrpt.boman_index()
    seq_df['PHYC|Boman Index'] = descrpt.descriptor.reshape(-1)

    return seq_df


'''
Insert Amino acid composition to the sequence data_frame
Input: sequence data_frame {IDx: Seq_x}
Output: data_frame of Peptide Seq {IDx: Seq_x, ..., AAC_Ax ... AAC_Yx}
'''


def insert_aac(seq_df):
    seq_df = seq_df.copy()
    # Compute AAC for peptide in specific A.A
    def get_aac(seq, aa):
        return seq.count(aa) / len(seq) * 100

    # processing data_frame
    data_size = seq_df.size
    for ll in _AALetter:
        seq_df['AAC|{}'.format(ll)] = list(map(get_aac, seq_df['Sequence'], [ll] * data_size))
    return seq_df






def insert_paac(seq_df, lamb=3, w=0.1):
    seq_df = seq_df.copy()
    fastas = [[idx, seq] for idx, seq in zip(seq_df['Id'], seq_df['Sequence'])]
    encoding = paac(fastas, lambdaValue=lamb, w=w)
    encoding = pd.DataFrame(encoding[1:], columns=encoding[0])
    seq_df = pd.concat([seq_df, encoding.iloc[:, 1:]], axis=1)
    return seq_df




def construct_features(seq_df, paaclamb=3, paacw=0.5):
    """
    Construct Features for the AVPIden. We first investigated physiochemical feautres, AAC features, DiC features, 
    CKSAAGP features, PAAC features, and PHYC features.
    Parameters are pre-set according to the sequence identities.
    """
    seq_df = insert_aac(seq_df)
    seq_df = insert_paac(seq_df, lamb=paaclamb, w=paacw)
    seq_df = insert_phycs(seq_df)

    return seq_df

<>:469: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:471: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:473: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:469: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:471: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:473: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/19/51xjwpd57m1g5zzk1p6ps9g40000gn/T/ipykernel_5123/4061532156.py:469: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if n is 1:
/var/folders/19/51xjwpd57m1g5zzk1p6ps9g40000gn/T/ipykernel_5123/4061532156.py:471: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif n is 2:
/var/folders/19/51xjwpd57m1g5zzk1p6ps9g40000gn/T/ipykernel_5123/4061532156.py:473: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif n is 3:


In [12]:
def write_fasta(df, file_path, abbr_columns=None):
    """
    Save dataframe to a .fasta file, the df should contain at least columns named "Id" and "Sequence"
    
    df: dataframe for saving .fasta
    file_path: path(string) for saving the fasta file
    abbr_columns: string columns for adding abbreviations. Multiple abbr are splited by '|'.
    """
    Seqrecords = [SeqIO.SeqRecord(id=row['Id'], 
                              seq=Seq.Seq(row['Sequence']), 
                              description='|'.join(row[abbr_columns] if abbr_columns is not None else "")) \
             for idn, row in df.iterrows()]
    with open(file_path, 'w+') as fhandle:
        SeqIO.write(Seqrecords, fhandle, "fasta-2line")
        print("Saved {:d} sequences.".format(len(Seqrecords)))


def read_fasta(file_path):
    f = open(file_path, 'r', encoding='utf-8')
    fasta_list = np.array(f.readlines())
    # print(fasta_list)
    new_fasta = []
    for flag in range(0, len(fasta_list), 2):
        fasta_str = [fasta_list[flag].strip('\n').strip(), fasta_list[flag + 1].strip('\n').strip()]
        new_fasta.append(fasta_str)
    # print(new_fasta)
    seq_df = pd.DataFrame(data=new_fasta, columns=['Id', 'Sequence'])
    # print(new_fasta1.shape)
    return seq_df

In [17]:
seq_fasta = read_fasta("non_THP.txt")
seq_feature = construct_features(seq_fasta)
seq_feature.to_csv("non_THP_AAC_PAAC_PHYC.csv")

In [18]:
seq_feature

,Id,Sequence,AAC|A,AAC|C,AAC|D,AAC|E,AAC|F,AAC|G,AAC|H,AAC|I,...,PAAC|lambda3,PHYC|IEP,PHYC|Net Charge,PHYC|Hydrophobic Moment,PHYC|Hydrophobicity,PHYC|Transmembrane Propensity,PHYC|Aromacity,PHYC|Alpha Helical Propensity,PHYC|Aliphatic Index,PHYC|Boman Index
0,>ACP203,KAKLF,20.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000,...,0.327496,10.002737,1.758104,0.316465,0.240000,-0.548000,0.200000,1.224000,98.000000,0.278000
1,>ACP217,ACSAG,40.000000,20.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,...,0.138602,5.561548,-0.214026,0.284292,-0.340000,-0.052000,0.000000,1.014000,40.000000,-0.488000
2,>ACP259,WALAL,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.404608,5.525000,-0.239898,0.183105,-1.600000,1.186000,0.200000,1.234000,196.000000,-3.158000
3,>ACP279,FAKLA,40.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000,...,0.027398,8.750052,0.759103,0.714502,-0.460000,0.220000,0.200000,1.236000,118.000000,-1.194000
4,>ACP293,PPPEE,0.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.304677,4.240666,-2.030613,0.196176,1.200000,-2.024000,0.000000,0.888000,0.000000,2.724000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,>AMP270,RIGVLLARLPKLFSLFKLMGKKV,4.347826,0.000000,0.000000,0.000000,8.695652,8.695652,0.000000,4.347826,...,0.197175,11.999968,5.756086,0.339831,-0.178261,0.009565,0.086957,1.077826,148.260870,0.040000
647,>AMP341,LLDCWVRLGRYLLRRLKTPFTRL,0.000000,4.347826,4.347826,0.000000,4.347826,4.347826,0.000000,0.000000,...,0.219064,11.437223,4.749273,0.694884,-0.134783,-0.171739,0.130435,1.030000,131.304348,2.093043
648,>AMP355,SNMIEGVFAKGFKKASHLFKGIG,8.695652,0.000000,0.000000,4.347826,13.043478,17.391304,4.347826,8.695652,...,0.225493,10.000416,2.546135,0.458631,-0.060870,-0.273913,0.130435,1.033478,72.173913,0.419130
649,>AMP408,FALALKALKKALKKLKKALKKAL,26.086957,0.000000,0.000000,0.000000,4.347826,0.000000,0.000000,0.000000,...,0.190572,10.903296,8.751111,0.639309,0.386957,-0.614783,0.043478,1.260000,144.782609,0.072609
